In [23]:
# Example 19-2. osconfeed.py: Downloading osconfeed.json.
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'files/osconfeed.json'


def load( JSON):
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)  # <1>
        with urlopen(URL) as remote, open(JSON, 'wb') as local:  # <2>
            local.write(remote.read())

    with open(JSON) as fp:
        return json.load(fp)  # <3>

feed = load('files/osconfeed.json')
print("feed : \n", feed, "\n")
print("sorted(feed['Schedule'].keys()) : ", sorted(feed['Schedule'].keys()))
print("\nlen(value), key :")
for key, value in sorted(feed['Schedule'].items()):
    print('{:3} {}'.format(len(value), key))
print()
print("feed['Schedule']['speakers'][-1]['serial'] = ", feed['Schedule']['speakers'][-1]['serial'])
print("feed['Schedule']['events'][0]['name'] = ", feed['Schedule']['events'][0]['name'])
print("feed['Schedule']['events'][0]['speakers]' = ", feed['Schedule']['events'][0]['speakers'])

feed : 
 {'Schedule': {'events': [{'description': 'Aside from the fact that high school programming...', 'serial': 34505, 'time_stop': '2014-07-23 12:10:00', 'speakers': [157509], 'categories': ['Education'], 'venue_serial': 1462, 'name': 'Why Schools Don´t Use Open Source to Teach Programming', 'event_type': '40-minute conference session', 'website_url': 'http://oscon.com/oscon2014/public/schedule/detail/34505', 'time_start': '2014-07-23 11:30:00'}], 'venues': [{'name': 'F151', 'serial': 1462, 'category': 'Conference Venues'}], 'speakers': [{'twitter': 'sharewaveteam', 'serial': 157509, 'position': 'CTO', 'name': 'Robert Lefkowitz', 'url': 'http://sharewave.com/', 'photo': None, 'bio': 'Robert ´r0ml´ Lefkowitz is the CTO at Sharewave, a startup...', 'affiliation': 'Sharewave'}], 'conferences': [{'serial': 115}]}} 

sorted(feed['Schedule'].keys()) :  ['conferences', 'events', 'speakers', 'venues']

len(value), key :
  1 conferences
  1 events
  1 speakers
  1 venues

feed['Schedule']['

In [35]:
# Example 19-4. FrozenJSON from Example 19-5 allows reading attributes like name 
# and calling methods like .keys() and .items().
import sys
sys.path.append('/home/kwol/git/kw/jupyter/Fluent Python/module')

from collections import abc
import keyword

class FrozenJSON:
    """A read-only façade for navigating a JSON-like object
       using attribute notation
    """
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):
                key += '_'
            self.__data[key] = value  # <1>
    def __getattr__(self, name):  # <2>
        if hasattr(self.__data, name):
            return getattr(self.__data, name)  # <3>
        else:
            return FrozenJSON.build(self.__data[name])  # <4>
    @classmethod
    def build(cls, obj):  # <5>
        if isinstance(obj, abc.Mapping):  # <6>
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):  # <7>
            return [cls.build(item) for item in obj]
        else:  # <8>
            return obj

from osconfeed import load
raw_feed = load('files/osconfeed.json')
feed = FrozenJSON(raw_feed)
print("len(feed.Schedule.speakers) : ", len(feed.Schedule.speakers), "\n")
for key, value in sorted(feed.Schedule.items()):
    print('{:3} {}'.format(len(value), key))
print("\nfeed.Schedule.speakers[-1].name : ", feed.Schedule.speakers[-1].name)
talk = feed.Schedule.events[40]
print("talk : ", talk)
print("type(talk) : ", type(talk))
print("talk.name : ", talk.name)
print("talk.speakers : ", talk.speakers)

len(feed.Schedule.speakers) :  357 

  1 conferences
494 events
357 speakers
 53 venues

feed.Schedule.speakers[-1].name :  Carina C. Zona
talk :  <__main__.FrozenJSON object at 0x7fd6b6697e48>
type(talk) :  <class '__main__.FrozenJSON'>
talk.name :  There *Will* Be Bugs
talk.speakers :  [3471, 5199]


In [32]:
# Example 19-4. - continued
print("talk.flavor : ", talk.flavor)

KeyError: 'flavor'

In [ ]:
# Example 19-7. explore2.py: using __new__ instead of build to construct new objects
# that may or may not be instances of FrozenJSON.
from collections import abc

class FrozenJSON:
    """A read-only façade for navigating a JSON-like object
       using attribute notation
    """
    def __new__(cls, arg):  # <1>
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls)  # <2>
        elif isinstance(arg, abc.MutableSequence):  # <3>
            return [cls(item) for item in arg]
        else:
            return arg
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if iskeyword(key):
                key += '_'
            self.__data[key] = value
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON(self.__data[name])  # <4>

In [43]:
# Example 19-9. schedule1.py: exploring OSCON schedule data saved to a shelve.Shelf.
import sys
sys.path.append('/home/kwol/git/kw/jupyter/Fluent Python/module')

import warnings
import osconfeed  # <1>

DB_NAME = 'files/schedule1_db'
CONFERENCE = 'conference.115'

class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)  # <2> !!! build an instance with attributes created from keyword arguments 

def load_db(db):
    raw_data = osconfeed.load('files/osconfeed.json')  # <3>
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():  # <4>
        record_type = collection[:-1]  # <5>
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])  # <6>
            record['serial'] = key  # <7>
            db[key] = Record(**record)  # <8>

import shelve
db = shelve.open(DB_NAME)
if CONFERENCE not in db:
    load_db(db)
speaker = db['speaker.3471']
print("type(speaker) : ", type(speaker))
print("speaker.name, speaker.twitter : ",  speaker.name, ",", speaker.twitter)
db.close()

type(speaker) :  <class '__main__.Record'>
speaker.name, speaker.twitter :  Anna Martelli Ravenscroft , annaraven


In [48]:
# Example 19-15. bulkfood_v1.py: the simplest LineItem class
class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

raisins = LineItem('Golden raisins', 10, 6.95)
print("raisins.subtotal() : ", raisins.subtotal())
raisins.weight = -20
print("raisins.subtotal() : ", raisins.subtotal())

raisins.subtotal() :  69.5
raisins.subtotal() :  -139.0


In [49]:
# Example 19-17. bulkfood_v2.py: a LineItem with a weight property.
class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight  # <1>
        self.price = price
    def subtotal(self):
        return self.weight * self.price
    @property  # <2>
    def weight(self):  # <3>
        return self.__weight  # <4>
    @weight.setter  # <5>
    def weight(self, value):
        if value > 0:
            self.__weight = value  # <6>
        else:
            raise ValueError('value must be > 0')  # <7>
            
walnuts = LineItem('walnuts', 0, 10.00)

ValueError: value must be > 0

In [ ]:
# Example 19-18. bulkfood_v2b.py: same as Example 19-17 but without using decorators.
class LineItem:

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

    def get_weight(self):  # <1>
        return self.__weight

    def set_weight(self, value):  # <2>
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')

    weight = property(get_weight, set_weight)  # <3> Build the property and assign it to a public class attribute.

In [3]:
# Example 19-19 -20. Instance attribute shadows class data attribute
class Class: #
    data = 'the class data attr'
    @property
    def prop(self):
        return 'the prop value'

obj = Class()
print("vars(obj) : ", vars(obj))   # no instance attributes
print("obj.data : ", obj.data)   # retrieves the value of Class.data
print("Class.data : ", Class.data)
print()
obj.data = 'bar'
print("vars(obj) : ", vars(obj))
print("obj.data : ", obj.data)   # the instance data shadows the class dat
print("Class.data : ", Class.data)
print()
print("Class.prop : ", Class.prop)
print("obj.prop : ", obj.prop)
obj.prop = 'foo'

vars(obj) :  {}
obj.data :  the class data attr
Class.data :  the class data attr

vars(obj) :  {'data': 'bar'}
obj.data :  bar
Class.data :  the class data attr

Class.prop :  <property object at 0x7fdd501f3278>
obj.prop :  the prop value


AttributeError: can't set attribute

In [4]:
# Example 19 -20 - continued
obj.__dict__['prop'] = 'foo'  # Putting 'prop' directly in the obj.__dict__ works.
print("vars(obj) : ", vars(obj))  
print("obj.prop : ", obj.prop)
print()
Class.prop = 'baz'
print("Class.prop : ", Class.prop)
print("obj.prop : ", obj.prop, " - !!! BUG: in book out: 'the prop value'")  # <> !!! BUG

vars(obj) :  {'data': 'bar', 'prop': 'foo'}
obj.prop :  the prop value

Class.prop :  baz
obj.prop :  foo  - !!! BUG: in book out: 'the prop value'


In [66]:
# Example 19-21. New class property shadows existing instance attribute (continued from Example 19-20)
print("obj.data : ", obj.data)
print("Class.data : ", Class.data)
print()
Class.data = property(lambda self: 'the "data" prop value')
print("obj.data : ", obj.data)
del Class.data
print("obj.data : ", obj.data)

obj.data :  bar
Class.data :  the class data attr

obj.data :  the "data" prop value
obj.data :  bar


In [68]:
# Example 19-22. Documentation for a property
class Foo:
    @property
    def bar(self):
        '''The bar attribute'''
        return self.__dict__['bar']
    @bar.setter
    def bar(self, value):
        self.__dict__['bar'] = value

help(Foo.bar)
help(Foo)

Help on property:

    The bar attribute

Help on class Foo in module __main__:

class Foo(builtins.object)
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  bar
 |      The bar attribute



In [74]:
# Example 19-23. bulkfood_v2prop.py: The quantity property factory in use
def quantity(storage_name):  # <1>
    def qty_getter(instance):  # <2>
        return instance.__dict__[storage_name]  # <3>
    def qty_setter(instance, value):  # <4>
        if value > 0:
            instance.__dict__[storage_name] = value  # <5>
        else:
            raise ValueError('value must be > 0')
    return property(qty_getter, qty_setter)  # <6>

class LineItem:
    weight = quantity('weight')  # <1>
    price = quantity('price')  # <2>
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight  # <3>
        self.price = price
    def subtotal(self):
        return self.weight * self.price  # <4>
    
nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)
print("nutmeg.weight, nutmeg.price :", nutmeg.weight, ",", nutmeg.price)
print("sorted(vars(nutmeg).items()) : ", sorted(vars(nutmeg).items()))

nutmeg.weight, nutmeg.price : 8 , 13.95
sorted(vars(nutmeg).items()) :  [('description', 'Moluccan nutmeg'), ('price', 13.95), ('weight', 8)]


In [18]:
# Example 19-26. blackknight.py: Inspired by the Black Knight character of “Monty Python and the Holy Grail"
class BlackKnight:
    def __init__(self):
        self.members = ['an arm', 'another arm', 'a leg', 'another leg']
        self.phrases = ["'Tis but a scratch.", "It's just a flesh wound.",
                        "I'm invincible!", "All right, we'll call it a draw."]
    @property
    def member(self):
        print('next member is:')
        return self.members[0]
    @member.deleter
    def member(self):
        text = 'BLACK KNIGHT (loses {})\n-- {}'
        print(text.format(self.members.pop(0), self.phrases.pop(0)))

knight = BlackKnight()
knight.member

next member is:


'an arm'

In [19]:
# Example 19-26. - continued
del knight.member
del knight.member
del knight.member
del knight.member

BLACK KNIGHT (loses an arm)
-- 'Tis but a scratch.
BLACK KNIGHT (loses another arm)
-- It's just a flesh wound.
BLACK KNIGHT (loses a leg)
-- I'm invincible!
BLACK KNIGHT (loses another leg)
-- All right, we'll call it a draw.
